In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package libfuse2:amd64.
(Reading database ... 22280 files and directories currently installed.)
Preparing to unpack .../libfuse2_2.9.7-1ubuntu1_amd64.deb ...
Unpacking libfuse2:amd64 (2.9.7-1ubuntu1) ...
Selecting previously unselected package fuse.
Preparing to unpack .../fuse_2.9.7-1ubuntu1_amd64.deb ...
Unpacking fuse (2.9.7-1ubuntu1) ...
Selecting previously unselected package google-drive-ocamlfuse.
Preparing to unpack .../google-drive-ocamlfuse_0.7.0-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Setting up libfuse2:amd64 (2.9.7-1ubuntu1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
Setting up fuse (2.9.7-1ubuntu1) ...
Setting up google-drive-ocamlfuse (0.7.0-0ubuntu1~ubuntu18.04.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleus

In [ ]:
import pandas as pd
import io

from google.colab import files
uploaded = files.upload()

In [ ]:
ls

adc.json                 dataset1_final_eval.csv    dataset2_interim_eval.csv
data_processed.csv       dataset1_interim_eval.csv  interim.csv
data_processed_test.csv  dataset2.csv               interim_results.csv
dataset1.csv             dataset2_final_eval.csv    sample_data/


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import os
print(os.listdir("../"))

['bin', 'home', 'sys', 'boot', 'srv', 'sbin', 'tmp', 'lib', 'var', 'lib64', 'media', 'usr', 'opt', 'proc', 'dev', 'etc', 'run', 'root', 'mnt', 'content', '.dockerenv', 'datalab', 'colabtools', 'tools']


In [ ]:
def analyze_tweet(tweet):
    result = {}
    result['MENTIONS'] = tweet.count('USER_MENTION')
    result['URLS'] = tweet.count('URL')
    result['POS_EMOS'] = tweet.count('EMO_POS')
    result['NEG_EMOS'] = tweet.count('EMO_NEG')
    tweet = tweet.replace('USER_MENTION', '').replace(
        'URL', '')
    words = tweet.split()
    result['WORDS'] = len(words)
    bigrams = get_bigrams(words)
    result['BIGRAMS'] = len(bigrams)
    return result, words, bigrams


def get_bigrams(tweet_words):
    bigrams = []
    num_words = len(tweet_words)
    for i in range(num_words - 1):
        bigrams.append((tweet_words[i], tweet_words[i + 1]))
    return bigrams


def get_bigram_freqdist(bigrams):
    freq_dict = {}
    for bigram in bigrams:
        if freq_dict.get(bigram):
            freq_dict[bigram] += 1
        else:
            freq_dict[bigram] = 1
    counter = Counter(freq_dict)
    return counter

In [ ]:
import sys
from nltk.stem.porter import PorterStemmer


def preprocess_word(word):
    # Remove punctuation
    word = word.strip('\'"?!,.():;')
    # Convert more than 2 letter repetitions to 2 letter
    # funnnnny --> funny
    word = re.sub(r'(.)\1+', r'\1\1', word)
    # Remove - & '
    word = re.sub(r'(-|\')', '', word)
    return word


def is_valid_word(word):
    return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)

def handle_emojis(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' EMO_POS ', tweet)
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', ' EMO_POS ', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)', ' EMO_POS ', tweet)
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', tweet)
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', tweet)
    return tweet

def preprocess_tweet(tweet):
    processed_tweet = []
    # Convert to lower case
    tweet = tweet.lower()
    # Replaces URLs with the word URL
    tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', tweet)
    # Replace @handle with the word USER_MENTION
    tweet = re.sub(r'@[\S]+', 'USER_MENTION', tweet)
    # Replaces #hashtag with hashtag
    tweet = re.sub(r'#(\S+)', r' \1 ', tweet)
    # Remove RT (retweet)
    tweet = re.sub(r'\brt\b', '', tweet)
    # Replace 2+ dots with space
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    # Strip space, " and ' from tweet
    tweet = tweet.strip(' "\'')
    # Replace emojis with either EMO_POS or EMO_NEG
    tweet = handle_emojis(tweet)
    # Replace multiple spaces with a single space
    tweet = re.sub(r'\s+', ' ', tweet)
    words = tweet.split()

    for word in words:
        word = preprocess_word(word)
        if is_valid_word(word):
            if use_stemmer:
                word = str(porter_stemmer.stem(word))
            processed_tweet.append(word)

    return ' '.join(processed_tweet)
  
  
def preprocess_csv(csv_file_name, processed_file_name, test_file=False):
    save_to_file = open(processed_file_name, 'w')

    with open(csv_file_name, 'r') as csv:
        lines = csv.readlines()
        total = len(lines)
        for i, line in enumerate(lines):
            tweet_id = line[:line.find(',')]
            if not test_file:
                line = line[1 + line.find(','):]
                positive = int(line[:line.find(',')])
            line = line[1 + line.find(','):]
            tweet = line
            processed_tweet = preprocess_tweet(tweet)
            if not test_file:
                save_to_file.write('%s,%d,%s\n' %
                                   (tweet_id, positive, processed_tweet))
            else:
                save_to_file.write('%s,%s\n' %
                                   (tweet_id, processed_tweet))
            
    save_to_file.close()
    print( '\nSaved processed tweets to: %s' % processed_file_name)
    return processed_file_name


if __name__ == '__main__':
   
    use_stemmer = False
    csv_file_name = 'dataset2.csv'
    processed_file_name = 'data_processed.csv'
    preprocess_csv(csv_file_name, processed_file_name, test_file=True)


Saved processed tweets to: data_processed.csv


In [ ]:
x = pd.read_csv('data_processed.csv')

In [ ]:
if __name__ == '__main__':
   
    use_stemmer = False
    csv_file_name = 'dataset2_interim_eval.csv'
    processed_file_name = 'data_processed_test.csv'
    preprocess_csv(csv_file_name, processed_file_name, test_file=True)


Saved processed tweets to: data_processed_test.csv


In [ ]:
train_df = pd.read_csv('data_processed.csv')
test_df = pd.read_csv('data_processed_test.csv')
train_df['sentiment'] = pd.read_csv('dataset2.csv')['sentiment']
train_df.columns=['a','text','sentiment']
test_df.columns=['a','text']

In [ ]:
print(len(train_df))
train_df.head()


3791


,a,text,sentiment
0,0,my hair is blue,1
1,1,im sweating my forthcoming trip to e3 if i can...,0
2,2,USER_MENTION im sorry i dont understand your l...,0
3,3,website2018 my cupcakes,1
4,4,reading buyology before bedtime great premise ...,0


In [ ]:
def feat(data):
    ment=[]
    url=[]
    pe=[]
    ne=[]
    bg=[]
    for i in range(len(data)):
        ment.append(analyze_tweet(data['text'].values[i])[0]['MENTIONS'])
        url.append(analyze_tweet(data['text'].values[i])[0]['URLS'])
        pe.append(analyze_tweet(data['text'].values[i])[0]['POS_EMOS'])
        ne.append(analyze_tweet(data['text'].values[i])[0]['NEG_EMOS'])
        bg.append(analyze_tweet(data['text'].values[i])[0]['BIGRAMS'])
    data['ment']=ment
    data['url']=url
    data['pe']=pe
    data['ne']=ne
    data['bg']=bg
    return data

In [ ]:
import pandas as pd

import numpy as np
import xgboost as xgb

import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D,GRU
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

embed_size = 500 
max_features = 90000 
maxlen = 100 

train_df=feat(train_df)
test_df=feat(test_df)

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_df.text))
train_X = tokenizer.texts_to_sequences(train_df.text)
test_X = tokenizer.texts_to_sequences(test_df.text)

## Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)
train_X = pd.concat([pd.DataFrame(train_X),train_df[['ment','pe','ne','url','bg']].reset_index(drop=True)],axis=1)
test_X = pd.concat([pd.DataFrame(test_X),train_df[['ment','pe','ne','url','bg']].reset_index(drop=True)],axis=1)
## Get the sentiment values
train_y = train_df['sentiment'].values

train_x,val_X,train_Y, val_Y = train_test_split(train_X,train_y
                                    , test_size=0.1, random_state=2018)


from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf = xgb.XGBClassifier()

clf.fit(train_x,train_Y)

from sklearn.metrics import accuracy_score
print(accuracy_score(val_Y,clf.predict(val_X)))


tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_df.text))
train_X = tokenizer.texts_to_sequences(train_df.text)
test_X = tokenizer.texts_to_sequences(test_df.text)

## Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)
train_X = pd.concat([pd.DataFrame(train_X),train_df[['ment','pe','ne','url','bg']].reset_index(drop=True)],axis=1)
test_X = pd.concat([pd.DataFrame(test_X),train_df[['ment','pe','ne','url','bg']].reset_index(drop=True)],axis=1)
## Get the sentiment values
train_y = train_df['sentiment'].values

train_x,val_X,train_Y, val_Y = train_test_split(train_X,train_y
                                    , test_size=0.1, random_state=2018)


from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf = xgb.XGBClassifier()
clf.fit(train_x,train_Y)

from sklearn.metrics import accuracy_score
print(accuracy_score(val_Y,clf.predict(val_X)))



train_df.head()

new_train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=2018)

## fill up the missing values
train_X = train_df["text"].fillna("_na_").values
new_train_X = new_train_df["text"].fillna("_na_").values
val_X = val_df["text"].fillna("_na_").values
test_X = test_df["text"].fillna("_na_").values

## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
new_train_X = tokenizer.texts_to_sequences(new_train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

## Pad the sentences 
new_train_X = pad_sequences(new_train_X, maxlen=maxlen)
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

## Get the sentiment values
new_train_y = new_train_df['sentiment'].values
train_y = train_df['sentiment'].values
val_y = val_df['sentiment'].values

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()

clf.fit(new_train_X,new_train_y)

from sklearn.metrics import accuracy_score
print(accuracy_score(val_y,clf.predict(val_X)))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.6421052631578947


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.6421052631578947
0.5


In [ ]:

def get_model():
    embed_size = 128
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Bidirectional(LSTM(50, return_sequences=True))(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(50, activation="relu")(x)    
    x = Dropout(0.1)(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model


model = get_model()

model.fit(new_train_X, new_train_y, batch_size=512, epochs=10, validation_data=(val_X, val_y))

Train on 3411 samples, validate on 380 samples
Epoch 1/10
3411/3411 [==============================] - 23s 7ms/step - loss: 0.6910 - acc: 0.5432 - val_loss: 0.6889 - val_acc: 0.5474
Epoch 2/10
3411/3411 [==============================] - 16s 5ms/step - loss: 0.6856 - acc: 0.5558 - val_loss: 0.6851 - val_acc: 0.5474
Epoch 3/10
3411/3411 [==============================] - 16s 5ms/step - loss: 0.6804 - acc: 0.5558 - val_loss: 0.6767 - val_acc: 0.5474
Epoch 4/10
3411/3411 [==============================] - 16s 5ms/step - loss: 0.6661 - acc: 0.5576 - val_loss: 0.6576 - val_acc: 0.5500
Epoch 5/10
3411/3411 [==============================] - 16s 5ms/step - loss: 0.6301 - acc: 0.6201 - val_loss: 0.6093 - val_acc: 0.6868
Epoch 6/10
3411/3411 [==============================] - 16s 5ms/step - loss: 0.5237 - acc: 0.7593 - val_loss: 0.6820 - val_acc: 0.7289
Epoch 7/10
3411/3411 [==============================] - 16s 5ms/step - loss: 0.4391 - acc: 0.8481 - val_loss: 0.6194 - val_acc: 0.6921
Epoch 8/

In [ ]:
model.fit(train_X, train_y, batch_size=512, epochs=15)

Epoch 1/15
3791/3791 [==============================] - 18s 5ms/step - loss: 0.3445 - acc: 0.8805
Epoch 2/15
3791/3791 [==============================] - 17s 5ms/step - loss: 0.3053 - acc: 0.9013
Epoch 3/15
3791/3791 [==============================] - 17s 5ms/step - loss: 0.2755 - acc: 0.9164
Epoch 4/15
3791/3791 [==============================] - 17s 5ms/step - loss: 0.2589 - acc: 0.9246
Epoch 5/15
3791/3791 [==============================] - 17s 5ms/step - loss: 0.2370 - acc: 0.9309
Epoch 6/15
3791/3791 [==============================] - 17s 5ms/step - loss: 0.2738 - acc: 0.9172
Epoch 7/15
3791/3791 [==============================] - 18s 5ms/step - loss: 0.4543 - acc: 0.7940
Epoch 8/15
3791/3791 [==============================] - 17s 5ms/step - loss: 0.3460 - acc: 0.9011
Epoch 9/15
3791/3791 [==============================] - 17s 4ms/step - loss: 0.2810 - acc: 0.9119
Epoch 10/15
3791/3791 [==============================] - 17s 4ms/step - loss: 0.2371 - acc: 0.9153
Epoch 11/15
3791/37

In [ ]:
pred = model.predict(test_X)

In [ ]:
interim = pd.read_csv('dataset2_interim_eval.csv')
interim = interim.drop('text', axis = 1)
interim.head()

,id
0,0
1,1
2,2
3,3
4,4


In [ ]:
interim['label'] = pred
interim.head()

,id,label
0,0,0.098074
1,1,0.994175
2,2,0.995501
3,3,0.994286
4,4,0.998858


In [ ]:
interim.loc[interim.label > 0.5 , 'label'] = 1
interim.loc[interim.label < 0.5 , 'label'] = 0

In [ ]:
interim.to_csv('interim_results.csv',index=None)
from google.colab import files
files.download('interim_results.csv')